# Obj: Generate insight about and predict the turnover of data scientists (not done yet :p)

__Motivation:__ A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates.

__Objective__:

Given features, namely:
- enrollee_id : Unique ID for candidate
- city: City code
- city_ development _index : Developement index of the city (scaled)
- gender: Gender of candidate
- relevent_experience: Relevant experience of candidate
- enrolled_university: Type of University course enrolled if any
- education_level: Education level of candidate
- major_discipline :Education major discipline of candidate
- experience: Candidate total experience in years
- company_size: No of employees in current employer's company
- company_type : Type of current employer
- last_new_job: Difference in years between previous job and current job
- training_hours: training hours completed,

generate insight and predict whether or not a data scientist will be looking for a new job.

__Datasets__:

The [data](https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists) is taken from kaggle, about which many notebooks and an academic [paper](10.24297/ijmit.v16i.9058) has been written.  

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("../datasets/aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


## Exploratory Data Analysis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

There are lots and lots of NA values.

In [6]:
df.describe()

,enrollee_id,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


### Analyzing the NA values

I'd like to see whether or not the existence of NA values for a given feature is related to other feature

In [60]:
df_NA_city = df[["city", "gender", "company_size"]].copy()
df_NA_city["gender"]= df["gender"].isna()
df_NA_city["company_size"]= df["company_size"].isna()
df_NA_city["counter"] = 1

#city-gender
df_NA_city_gender = df_NA_city.groupby(["city", "gender"])["counter"].sum()
df_NA_city_gender = df_NA_city_gender.unstack()
df_NA_city_gender["NA_proportion"] = df_NA_city_gender[True]/(df_NA_city_gender[False] + df_NA_city_gender[True])
df_NA_city_gender.sort_values("NA_proportion", ascending=False).head(15)

gender,False,True,NA_proportion
city,,,
city_79,1.0,7.0,0.875000
city_126,7.0,21.0,0.750000
city_129,1.0,2.0,0.666667
city_121,1.0,2.0,0.666667
city_33,7.0,10.0,0.588235
city_42,6.0,7.0,0.538462
city_134,21.0,22.0,0.511628
city_107,3.0,3.0,0.500000
city_146,4.0,4.0,0.500000


In [59]:
#city-company_size
df_NA_city_comsize = df_NA_city.groupby(["city", "company_size"])["counter"].sum()
df_NA_city_comsize = df_NA_city_comsize.unstack()
df_NA_city_comsize["NA_proportion"] = df_NA_city_comsize[True]/(df_NA_city_comsize[False] + df_NA_city_comsize[True])
df_NA_city_comsize.sort_values("NA_proportion", ascending=False).head(15)

company_size,False,True,NA_proportion
city,,,
city_109,1.0,8.0,0.888889
city_79,1.0,7.0,0.875000
city_180,1.0,6.0,0.857143
city_139,1.0,4.0,0.800000
city_48,3.0,10.0,0.769231
city_127,3.0,7.0,0.700000
city_106,3.0,6.0,0.666667
city_107,2.0,4.0,0.666667
city_134,15.0,28.0,0.651163
